In [1]:
import pandas as pd

In [2]:
file = pd.read_csv('events_up_to_01062018.csv',low_memory=False)

In [3]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341681 entries, 0 to 2341680
Data columns (total 23 columns):
timestamp                   object
event                       object
person                      object
url                         object
sku                         float64
model                       object
condition                   object
storage                     object
color                       object
skus                        object
search_term                 object
staticpage                  object
campaign_source             object
search_engine               object
channel                     object
new_vs_returning            object
city                        object
region                      object
country                     object
device_type                 object
screen_resolution           object
operating_system_version    object
browser_version             object
dtypes: float64(1), object(22)
memory usage: 410.9+ MB


In [11]:
def isnew (x):
    c = x['new_vs_returning'].value_counts().index
    return ('New' in c)
new = file.groupby('person').apply(isnew)

In [14]:
new.value_counts()

True     38242
False      587
dtype: int64

In [17]:
def region (x):
    c = x['region'].value_counts().index
    if len(c)>0:
        return c[0]
    else:
        return 'Unknown'
reg = file.groupby('person').apply(region)

In [19]:
reg.head()

person
0008ed71              Unknown
00091926    Rio Grande do Sul
00091a7a         Minas Gerais
000ba417            Sao Paulo
000c79fe         Minas Gerais
dtype: object

In [22]:
def viocelulares(x):
    return (x.loc[x.event=='viewed product']['event'].size > 0)
viocelulares = file.groupby('person').apply(viocelulares)

In [23]:
viocelulares.value_counts()

True     37130
False     1699
dtype: int64

In [30]:
features = pd.DataFrame(viocelulares).join(pd.DataFrame(reg),lsuffix='v').join(pd.DataFrame(new))
features.columns = ['vio_celulares','region','is_new']

In [31]:
features.head()

,vio_celulares,region,is_new
person,,,
0008ed71,False,Unknown,True
00091926,True,Rio Grande do Sul,True
00091a7a,True,Minas Gerais,True
000ba417,True,Sao Paulo,True
000c79fe,True,Minas Gerais,True
